# Google Map Scraping

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from parsel import Selector                                                      #for the HTML or XML text that you want to parse:
import urllib.request
import time
import re
from datetime import datetime

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from progressbar import ProgressBar                                               # to have a progress bar
import glob


import warnings
warnings.filterwarnings("ignore")

In [2]:
# Call the Chrome Driver and activate it.
driver = webdriver.Chrome(executable_path = r'chromedriver')

In [3]:
search_terms= ["diagnostic imaging"]

markets = ['St Louis County Missouri']


In [4]:
#Obtain information for Businesses from google map
df_level_1 = pd.DataFrame(columns=["LocName","LocHREF"])
pbar = ProgressBar()
N = 20
for market in pbar(markets):
    t1 = pd.Timestamp.now()
    market = market.replace(' ',"+")
    print(market)
    for search_string in search_terms:
        search_string = search_string.replace(' ',"+")
        print(search_string)

        url_local = search_string + '+' + market
        url_local = 'https://www.google.com/maps/search/' + url_local
        time.sleep(0.5)
        print(url_local)
        driver.get(url_local)
        driver.maximize_window()
        time.sleep(0.5)

        keep_looping = True
        page_number = 0
        while page_number <= N:
            for i in range(100):
                # Scroll page down
                try:
                    driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[{}]/div/a'.format(8*(i+1)+1)).send_keys(Keys.END)
                    time.sleep(0.5)
                except:
                    print("End of search bar")
                break
            
            page_content = driver.page_source
            # print(page_content)
            response = Selector(page_content)
            idx = 0
            for el in response.xpath('//div[contains(@aria-label, "Results for")]/div/div[./a]'):

                LocHREF = el.xpath('./a/@href').extract_first('')
                df_level_1.loc[idx,"LocHREF"] = LocHREF

                LocName= el.xpath('./a/@aria-label').extract_first('')
                df_level_1.loc[idx,"LocName"] = LocName
                
                # df_level_1.loc[idx,"market"] = market
                # df_level_1.loc[idx,"search_string"] = search_string  
                
                idx += 1

            page_number += 1
        

        df_level_1.to_csv("google_map_result_{}_{}.csv".format(market, pd.datetime.today().strftime('%y%m%d-%H%M%S') ),index = False)#,int(page_number)))              
        # driver.quit();
                
df_level_1

St+Louis+County+Missouri
diagnostic+imaging
https://www.google.com/maps/search/diagnostic+imaging+St+Louis+County+Missouri


100% |########################################################################|


,LocName,LocHREF
0,St. Luke's RAYUS Radiology,https://www.google.com/maps/place/St.+Luke's+R...
1,Metro Imaging South County,https://www.google.com/maps/place/Metro+Imagin...
2,Professional Imaging,https://www.google.com/maps/place/Professional...
3,St. Louis Imaging,https://www.google.com/maps/place/St.+Louis+Im...
4,Excel Imaging,https://www.google.com/maps/place/Excel+Imagin...
...,...,...
115,"Dr. Mark A. Chambers, MD",https://www.google.com/maps/place/Dr.+Mark+A.+...
116,"Dr. David W. Wu, MD",https://www.google.com/maps/place/Dr.+David+W....
117,"Dr. Thomas E. Reh, MD",https://www.google.com/maps/place/Dr.+Thomas+E...
118,Mercy Breast Center - Clayton-Clarkson Suite 110,https://www.google.com/maps/place/Mercy+Breast...


In [8]:
# for f in glob.glob('google_map_result_*.csv'):
#     data = pd.read_csv(f)

# data

def pd_read_pattern(pattern):
    files = glob.glob(pattern)

    df = pd.DataFrame()
    for f in files:
        df = df.append(pd.read_csv(f))

    return df.reset_index(drop=True)

data = pd_read_pattern('google_map_result_*.csv')


In [9]:
data

,LocName,LocHREF
0,St. Luke's RAYUS Radiology,https://www.google.com/maps/place/St.+Luke's+R...
1,Metro Imaging South County,https://www.google.com/maps/place/Metro+Imagin...
2,Professional Imaging,https://www.google.com/maps/place/Professional...
3,St. Louis Imaging,https://www.google.com/maps/place/St.+Louis+Im...
4,Excel Imaging,https://www.google.com/maps/place/Excel+Imagin...
...,...,...
115,"Dr. Mark A. Chambers, MD",https://www.google.com/maps/place/Dr.+Mark+A.+...
116,"Dr. David W. Wu, MD",https://www.google.com/maps/place/Dr.+David+W....
117,"Dr. Thomas E. Reh, MD",https://www.google.com/maps/place/Dr.+Thomas+E...
118,Mercy Breast Center - Clayton-Clarkson Suite 110,https://www.google.com/maps/place/Mercy+Breast...


In [7]:
data.shape

(0, 0)

In [10]:
# Obtain business information for each individual location
for i in range(20):#len(data)):
    t = pd.Timestamp.now()
    url = data.loc[i, "LocHREF"]#"https://www.google.com/maps/place/CORA+Physical+Therapy+South+County/data=!4m6!3m5!1s0x87d8cf35f7d3f769:0x7b3019c994319873!8m2!3d38.5039686!4d-90.3788281!16s%2Fg%2F1pp2tvx8w?authuser=0&hl=en&rclk=1"
    driver.get(url)
    time.sleep(1)
    try:
        Name = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]').text
        data.loc[i,"Name"] = Name
    except:
        pass
    
    try:
        Business = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[2]/span[1]/span[1]/button').text
        data.loc[i,"Business_b"] = Business
    except:
        pass
    
    try:
        Address  = driver.find_element_by_xpath('//button[contains(@ aria-label,"Address")]').text
        data.loc[i,"Address"] = Address
    except:
        pass
    
    try:
        AddressDetail = driver.find_element_by_xpath('//div[contains(@ aria-label,"Located in")]').text
        data.loc[i,"AddressDetail"] = AddressDetail
    except:
        pass
    
    try:
        WebSite = driver.find_element_by_xpath('//a[contains(@ aria-label,"Website")]').text
        data.loc[i,"WebSite"] = WebSite
    except:
        pass
    
    print(i,"\t",t, data.loc[i,"LocName"],Name, Business, Address , WebSite)    
    data.to_csv("google_map_result_{}_{}_B.csv".format(market,search_string),index = False)
    
data.head(5)

0 	 2022-09-28 14:59:31.487299 St. Luke's RAYUS Radiology St. Luke's RAYUS Radiology Medical diagnostic imaging center 10333-A Clayton Rd, Frontenac, MO 63131 rayusradiology.com
1 	 2022-09-28 14:59:35.525519 Metro Imaging South County Metro Imaging South County Medical diagnostic imaging center 13303 Tesson Ferry Rd #30, St. Louis, MO 63128 metroimaging.org
2 	 2022-09-28 14:59:39.990127 Professional Imaging Professional Imaging MRI center 777 S New Ballas Rd # 5, St. Louis, MO 63141 pi-imaging.com
3 	 2022-09-28 14:59:43.326880 St. Louis Imaging St. Louis Imaging MRI center 5000 Cedar Plaza Pkwy #110, St. Louis, MO 63128 pi-imaging.com
4 	 2022-09-28 14:59:46.938057 Excel Imaging Excel Imaging Medical diagnostic imaging center 9930 Watson Rd, St. Louis, MO 63126 essehealth.com
5 	 2022-09-28 14:59:50.042663 Diagnostic Imaging Group Diagnostic Imaging Group Medical diagnostic imaging center 232 S Woods Mill Rd, Chesterfield, MO 63017 stlukes-stl.com
6 	 2022-09-28 14:59:54.858464 Metr

,LocName,LocHREF,Name,Business_b,Address,WebSite
0,St. Luke's RAYUS Radiology,https://www.google.com/maps/place/St.+Luke's+R...,St. Luke's RAYUS Radiology,Medical diagnostic imaging center,"10333-A Clayton Rd, Frontenac, MO 63131",rayusradiology.com
1,Metro Imaging South County,https://www.google.com/maps/place/Metro+Imagin...,Metro Imaging South County,Medical diagnostic imaging center,"13303 Tesson Ferry Rd #30, St. Louis, MO 63128",metroimaging.org
2,Professional Imaging,https://www.google.com/maps/place/Professional...,Professional Imaging,MRI center,"777 S New Ballas Rd # 5, St. Louis, MO 63141",pi-imaging.com
3,St. Louis Imaging,https://www.google.com/maps/place/St.+Louis+Im...,St. Louis Imaging,MRI center,"5000 Cedar Plaza Pkwy #110, St. Louis, MO 63128",NaN
4,Excel Imaging,https://www.google.com/maps/place/Excel+Imagin...,Excel Imaging,Medical diagnostic imaging center,"9930 Watson Rd, St. Louis, MO 63126",essehealth.com


In [11]:
data['ZipCode']=data['Address'].str.extract('\,\s\w{2}\s(\d*)')
data['State'] = data['Address'].str.extract('\,\s(\w{2})\s\d*')
data['Lat'] = data['LocHREF'].str.extract('!\d{1}\w{1}(\d{2}.\d+)')
data['Long'] = data['LocHREF'].str.extract('!\d{1}\w{1}(-\d{2}.\d+)')

In [12]:
data

,LocName,LocHREF,Name,Business_b,Address,WebSite,ZipCode,State,Lat,Long
0,St. Luke's RAYUS Radiology,https://www.google.com/maps/place/St.+Luke's+R...,St. Luke's RAYUS Radiology,Medical diagnostic imaging center,"10333-A Clayton Rd, Frontenac, MO 63131",rayusradiology.com,63131,MO,38.6332855,-90.4071807
1,Metro Imaging South County,https://www.google.com/maps/place/Metro+Imagin...,Metro Imaging South County,Medical diagnostic imaging center,"13303 Tesson Ferry Rd #30, St. Louis, MO 63128",metroimaging.org,63128,MO,38.4834716,-90.3861504
2,Professional Imaging,https://www.google.com/maps/place/Professional...,Professional Imaging,MRI center,"777 S New Ballas Rd # 5, St. Louis, MO 63141",pi-imaging.com,63141,MO,38.6391915,-90.445811
3,St. Louis Imaging,https://www.google.com/maps/place/St.+Louis+Im...,St. Louis Imaging,MRI center,"5000 Cedar Plaza Pkwy #110, St. Louis, MO 63128",NaN,63128,MO,38.485171,-90.3823746
4,Excel Imaging,https://www.google.com/maps/place/Excel+Imagin...,Excel Imaging,Medical diagnostic imaging center,"9930 Watson Rd, St. Louis, MO 63126",essehealth.com,63126,MO,38.5560757,-90.3948507
...,...,...,...,...,...,...,...,...,...,...
115,"Dr. Mark A. Chambers, MD",https://www.google.com/maps/place/Dr.+Mark+A.+...,NaN,NaN,NaN,NaN,NaN,NaN,38.6455339,-90.4462228
116,"Dr. David W. Wu, MD",https://www.google.com/maps/place/Dr.+David+W....,NaN,NaN,NaN,NaN,NaN,NaN,38.555947,-90.3946858
117,"Dr. Thomas E. Reh, MD",https://www.google.com/maps/place/Dr.+Thomas+E...,NaN,NaN,NaN,NaN,NaN,NaN,38.632997,-90.3111919
118,Mercy Breast Center - Clayton-Clarkson Suite 110,https://www.google.com/maps/place/Mercy+Breast...,NaN,NaN,NaN,NaN,NaN,NaN,38.6054548,-90.5883044


In [ ]:
# Obtain latitude and longitude for each Location
url = 'https://www.mapdevelopers.com/geocode_tool.php'
driver.get(url)
for i in range(len(data)):
    t = pd.Timestamp.now()
    url = 'https://www.mapdevelopers.com/geocode_tool.php'
    x = data.loc[i, 'Address']
    try:
        what_search_box = driver.find_element_by_id("address")
        what_search_box.clear()
        what_search_box.send_keys(x)
        what_search_box.send_keys(Keys.ENTER)
        time.sleep(2)    

        data.loc[i,"Lat"] = driver.find_element_by_id('display_lat').text
        data.loc[i,"Long"] = driver.find_element_by_id('display_lng').text
    except:
        pass
    print(i,pd.Timestamp.now() - t, data.loc[i,['Lat','Long']])
    data.to_csv("google_map_result_{}_{}_c.csv".format(market,search_string),index = False)
#     print(i, df['zipcode'], t, pd.Timestamp.now() - t)

In [ ]:
data['ZipCode']=data['Address'].str[-5:].str.strip().astype(int)
data['State']  =data['Address'].str[-8:-6]
data

In [ ]:
county = pd.read_excel(r'C:\Users\Dshahabi\Documents\SampleWorks\HMSA\2019ZipMkt.xlsx')
county["zipcode"]=county["zipcode"].astype(int)
county = county.set_index("zipcode")['County'].to_dict()
data["County"] = data["ZipCode"].map(county)


In [ ]:
data

In [ ]:
data.to_csv("google_map_result_{}_{}_d.csv".format(market,search_string),index = False)


In [ ]:
End!